In [1]:
#Convert YOLO predictions to format needed for Njobvu-AI input

import pandas as pd
import json
import yaml

In [32]:
#define conversion function           **change path format below**
def convert_entry(entry, class_ids, img_dims):
    class_id = entry["category_id"]
    class_name = class_ids.get(class_id, "unknown")
    dimensions = img_dims[img_dims['image_name'] == entry['image_id'] + '.JPG']

    converted_entry = {
        "frame_id": 1,
        # "filename": f"/data/{entry['image_id']}.JPG",  #for finding files locally
        "filename": f"/nfs6/FW_HMSC/Levi_Lab/YOLO_photos/COA_training/oregon_critters/demo2/{dimensions['full_path'].values[0]}", #for files on server
        "objects": []
    }
    
    #to handle potentially empty dimensions
    if not dimensions.empty:
        converted_entry["objects"].append({
                "class_id": int(class_id),
                "name": class_name,
                "image_width": int(dimensions['img_width'].values[0]),
                "image_height": int(dimensions['img_height'].values[0]),
                "relative_coordinates": {
                    "center_x": float(entry["bbox"][0] / dimensions['img_width'].values[0]),  #check math here
                    "center_y": float(entry["bbox"][1] / dimensions['img_height'].values[0]),  
                    "width": float(entry["bbox"][2] / dimensions['img_width'].values[0]), 
                    "height": float(entry["bbox"][3] / dimensions['img_height'].values[0]),  
                },
                "confidence": float(entry["score"])
            })
    else:
        print(f"No dimensions found for image ID: {entry['image_id']}")

    return converted_entry


In [29]:
#specify file paths for YOLO predictions, YAML with class IDs, and CSV with image dimensions
yolo_path = '/Volumes/Cara_cam_traps/CV4E/data_cleaned/demo2/predictions.json'
yaml_path = '/Volumes/Cara_cam_traps/CV4E/data_cleaned/demo2/dataset.yaml'
csv_path = '/Volumes/Cara_cam_traps/CV4E/data_cleaned/demo2/demo2_with_empties.csv'

#load these files
with open(yolo_path, 'r') as json_file:
    yolo_data = json.load(json_file)

with open(yaml_path, 'r') as yaml_file:
    yaml_data = yaml.safe_load(yaml_file)
    class_ids = yaml_data.get('names', {})

img_dims = pd.read_csv(csv_path)

In [33]:
# Convert each entry in the original JSON to the new format
converted_json = [convert_entry(entry, class_ids, img_dims) for entry in yolo_data]

# Output the converted JSON
#print(json.dumps(converted_json, indent=2))


In [34]:
#specify output locations
out_json = '/Volumes/Cara_cam_traps/CV4E/data_cleaned/demo2/predictions_njobvu.json'
out_text = '/Volumes/Cara_cam_traps/CV4E/data_cleaned/demo2/predictions_njobvu.txt'

#save
with open(out_json, "w") as json_output_file:
    json.dump(converted_json, json_output_file, indent=2)

#also save as a text file
with open(out_text, "w") as text_output_file:
    text_output_file.write(json.dumps(converted_json, indent=2))
